In [1]:
import numpy as np
import pandas as pd
import sympy as sp
from fractions import Fraction
import itertools
import timeit

### Funciones previas

In [2]:
def permutation(lst):
    
    if len(lst) == 0:
        return []
    if len(lst) == 1:
        return [lst]
    
    l = []

    for i in range(len(lst)):
        m = lst[i]
        remLst = lst[:i] + lst[i+1:]

        for p in permutation(remLst):
            l.append([m] + p)
        
    return l

# Paridad de las permutaciones (solo funciona con el orden de la funcion anterior)

def par(lst):
    
    l = []
    
    for i in range(len(lst)):
        if i % 2 == 0:
            l.append(1)
            
        else:
            l.append(-1)
    return np.cumprod(np.array(l))

def combinations(n,k):
    
    u = list(range(0,n))
    
    c = [list(v) for v in itertools.product(u, repeat=k) if len([i for i in v if v.count(i)>1])==0]
    
    return c 
 
def kCombinations(output, temp, index, n, i, K):
    if index == K:
        output.append(temp.copy())
        return
    
    if i >=n:
        return
    
    temp[index] = i #+ 1
    kCombinations(output, temp, index + 1, n, i + 1, K)
    kCombinations(output, temp, index, n, i + 1, K)
    
def findKCombination(K,n):
    
    output = []
    temp = [0]*K
    kCombinations(output, temp, 0, n, 0, K)
    return output

### Delta Generalizada

In [3]:
def Delta(upI,lwI):
    
    per = permutation(lwI)
    pari = par(per)
    
    # Verifica si hay indices repetidos
    
    if len([i for i in upI if upI.count(i)>1]) > 0:
        return 0
    if len([i for i in lwI if lwI.count(i)>1]) > 0:
        return 0  
    
    # Busca si los indices superiores son una permutacion (par o impar) del los inferiores
    
    if upI in per:
        k = per.index(upI)
        return sp.Rational(pari[k],np.math.factorial(len(upI)))
    else:
        return 0

### Check de traza

Deberia ser 

$\Delta^{\mu_1 ... \mu_k}_{\nu_1 ... \nu_k} = \frac{1}{k!}\frac{n!}{(n-k)!}$

In [4]:
def TraDelta(n,k):
    u = 0
    for i in combinations(n,k):
        u = u + Delta(i,i)
    return u  

def checkDelta(n,k):
    
    s = sp.Rational(1,np.math.factorial(k))*sp.Rational(np.math.factorial(n),np.math.factorial(n-k))
    
    if TraDelta(n,k) == s:
        return True
    else: return False
        

In [5]:
checkDelta(4,1)

True

In [6]:
checkDelta(4,2)

True

In [7]:
checkDelta(4,3)

True

In [8]:
checkDelta(5,2)

True

In [9]:
checkDelta(5,3)

True

In [10]:
checkDelta(5,4)

True

### Representacion fundamental de SU(n)

In [11]:
def Li(n,i,j,a,b):
    
    s = Delta([a],[j])*Delta([i],[b])-Delta([i],[j])*Delta([a],[b])*sp.Rational(1,n)
    
    return s

def LiM(n,i,j):
    
    u = np.zeros((n,n),dtype=object)
    
    for a in range(0,n):
        for b in range(0,n):
            u[a][b]=Li(n,i,j,a,b)
    return u   

def L2(n,i,j):
    
    u = np.zeros((n,n),dtype=object)
    
    for a in range(0,n):
        for b in range(a,n):
            u[a][b] = (sp.Rational(1,2))*(Li(n,i,j,a,b)+Li(n,j,i,a,b))
            u[b][a] = u[a][b]
            
    return u          

def M2(n,i,k):
    
    u = np.zeros((n,n),dtype=object)
    
    for a in range(0,n):
        for b in range(a,n):
            u[a][b] = (sp.Rational(1,2))*(1j)*(Li(n,i,k,a,b)-Li(n,k,i,a,b))
            u[b][a] = -u[a][b]
            
    return u

def D(n,k):
    
    u = sp.sqrt(2*(k+1)*(k+2))
    
    s = np.zeros((n,n),dtype=object)
    
    for i in range(0,k+1):
        s = s + LiM(n,i,i)
        
    s = s - (k+1)*LiM(n,k+1,k+1)
    
    return s*(1/u)

In [12]:
def GenLamda3(n):
    
    p = []
    b = []

    for j in range(1,n):
        for i in range(0,j):
           # if i >= j:
           #     continue    

            p.append(2*L2(n,i,j))
            b.append('s')
            p.append(2*M2(n,i,j))
            b.append('as')
        p.append(2*D(n,j-1))
        b.append('d')
    return p,b

### Traza de los generadores $\lambda_I$

In [13]:
def Traza(A,B):
    
    u = np.trace(np.matmul(A,B,dtype=object))
    
    return u

In [14]:
def checkTL(Lamdas):
    
    u = (Lamdas[0].shape)[0]
    s = []
    
    for i,j in itertools.product(range(0,u*u-1), repeat=2):
        
        if Traza(Lamdas[i],Lamdas[j]) == 2*Delta([i],[j]):
            s.append(True)
        else:
            s.append(False)
    
    
    if all(s) == True:
        return 'Todas las relaciones se cumplen'
    else:
        return 'Algo va mal'
    
    

In [15]:
checkTL(GenLamda3(2)[0])

'Todas las relaciones se cumplen'

In [16]:
checkTL(GenLamda3(3)[0])

'Todas las relaciones se cumplen'

In [17]:
checkTL(GenLamda3(4)[0])

'Todas las relaciones se cumplen'

In [18]:
checkTL(GenLamda3(5)[0])

'Todas las relaciones se cumplen'

In [19]:
checkTL(GenLamda3(6)[0])

'Todas las relaciones se cumplen'

### Calculo de los $t_I$

$(t_I)^{\quad \ \ \  \mu_1 \ldots \mu_k}_{\nu_1 \ldots \nu_k} = (-1)^{k-1}(k-1)!\delta_{l_1}^{[\mu_1}\Delta^{\mu_2 \ldots \mu_k]\rho_1}_{\nu_1 \ldots \nu_k}(\lambda_I)_{\rho_1}^{l_1}$

Estos generadores satisfacen la condicion:

$Tr(t_It_J) = \frac{2(k-1)!(n-2)!}{k^2(n-k-1)!}\delta_{IJ}$

$[t_I,t_J]^{\quad \ \ \  \mu_1 \ldots \mu_k}_{\nu_1 \ldots \nu_k} = 2i\frac{(k-1)!}{k}f_{IJ}^K (t_K)^{\quad \ \ \  \mu_1 \ldots \mu_k}_{\nu_1 \ldots \nu_k} $

Resulta mas comodo definir todo como


$(t_I)^{\quad \ \ \  \mu_1 \ldots \mu_k}_{\nu_1 \ldots \nu_k} = (-1)^{k-1}\frac{2}{k}\delta_{l_1}^{[\mu_1}\Delta^{\mu_2 \ldots \mu_k]\rho_1}_{\nu_1 \ldots \nu_k}(\lambda_I)_{\rho_1}^{l_1}$


$Tr(t_It_J) = \frac{(n-2)!}{2(k-1)!(n-k-1)!}\delta_{IJ}$

$[t_I,t_J]^{\quad \ \ \  \mu_1 \ldots \mu_k}_{\nu_1 \ldots \nu_k} = if_{IJ}^K (t_K)^{\quad \ \ \  \mu_1 \ldots \mu_k}_{\nu_1 \ldots \nu_k} $


In [20]:
def Pro(UpI,LwI):
    
    s = 0
    
    perU = permutation(UpI[:len(UpI)-1])
    pari = par(perU)
    
    if len([i for i in UpI[:len(UpI)-1] if UpI[:len(UpI)-1].count(i)>1]) > 0:
        return 0
    else:
        for i in perU:
            k = perU.index(i)
            s = s + Delta([i[0]],[LwI[0]])*Delta(i[1:]+[UpI[-1]],LwI[1:])*pari[k]*Fraction(1,np.math.factorial(len(UpI)-1))
    return s 

In [21]:
def t3(n,matrix,upI,lwI):
    
    i = 0
    
    sig = ((-1)**(len(upI)-1))*sp.Rational(len(upI),2)
    
    s,t = np.where(matrix !=0 )
    
    
    for p,m in zip(s,t):
        UpI = upI + [p]
        LpI = [m] + lwI
        i = i + Pro(UpI,LpI)*(matrix)[p][m]
    
    return i*sig

### Verifiquemos la traza

La traza esta definida como:

$tr(t_It_J) = (t_I)_{\mu_1 \cdots \mu_k}^{\quad \ \ \nu_1 \cdots \nu_k}(t_J)_{\nu_1 \cdots \nu_k}^{\quad \ \ \mu_1 \cdots \mu_k}$

Se puede demostrar que 

$tr(t_It_J) = (k!)^2 (t_I)_{[\mu_1 \cdots \mu_k]}^{\quad \ \ [\nu_1 \cdots \nu_k]}(t_J)_{[\nu_1 \cdots \nu_k]}^{\quad \ \ [\mu_1 \cdots \mu_k]}$

Que es mucho mas eficiente a nivel de computo

In [22]:
def Tract3(matrix,k):
    
    u = (matrix[0].shape)[0]
    
    p = findKCombination(k,u)
    
    rs = sp.Rational(np.math.factorial(u-2),2*np.math.factorial(k-1)*np.math.factorial(u-k-1))
    
    t = []
    
    for i,j in itertools.product(range(0,u*u-1), repeat=2):
        
        l = []
        
        for upI in p:
            for lwI in p:
                l.append(t3(u,matrix[i],upI,lwI)*t3(u,matrix[j],lwI,upI))
                
  #      print(sp.nsimplify(sum(l)),sp.nsimplify(rs*Delta([i],[j])))
        
        if abs(np.math.factorial(k)**2*sum(l) - rs*Delta([i],[j])) < 1.0E-10:
            t.append(True)
        else:
            t.append(False)
            
    if all(t) == True:
        return 'Todas las relaciones se cumplen'
    else:
        return 'Algo va mal'     

In [23]:
Tract3(GenLamda3(3)[0],2)

'Todas las relaciones se cumplen'

In [24]:
Tract3(GenLamda3(4)[0],2)

'Todas las relaciones se cumplen'

In [25]:
Tract3(GenLamda3(4)[0],3)

'Todas las relaciones se cumplen'

In [26]:
Tract3(GenLamda3(5)[0],2)

'Todas las relaciones se cumplen'

In [27]:
Tract3(GenLamda3(5)[0],3)

'Todas las relaciones se cumplen'

In [28]:
Tract3(GenLamda3(5)[0],4)

'Todas las relaciones se cumplen'

### Veamos el bracket


In [29]:
def brat(n,A,B,upI,lwI):
    
    k = len(upI)
    u = 0
    
    p = combinations(n,k)
        
    for s in p:
        u = u + (t3(n,A,s,lwI)*t3(n,B,upI,s) - t3(n,B,s,lwI)*t3(n,A,upI,s))
        
    return u    
    

## Representacion Matricial de los generadores

In [30]:
def tM(n,Ma,st,k):
    
    values = findKCombination(k,n)
    dim = len(values)
    keys = [i for i in range(0,dim)]
    
    dic = dict(zip(keys,values))
    
    matrix = np.zeros((dim,dim),dtype=object)
    
    if st == 's':
        
        for i in range(0,dim):
            for j in range(i,dim):
                matrix[i][j] = np.math.factorial(k)*t3(n,Ma,dic[j],dic[i])
                matrix[j][i] = matrix[i][j]
                
    elif st == 'as':
        
        for i in range(0,dim):
            for j in range(i,dim):
                matrix[i][j] = np.math.factorial(k)*t3(n,Ma,dic[j],dic[i])
                matrix[j][i] = -matrix[i][j]
                
    elif st == 'd':
        
        for i in range(0,dim):
            matrix[i][i] = np.math.factorial(k)*t3(n,Ma,dic[i],dic[i])       
            
    return matrix

In [31]:
def tMf(n,k):
    
    ##############################################################
    ############ Todas las matrices y su simetria ################
    ##############################################################
    
    Lamda, Sim = GenLamda3(n)
    
    ###############################################################
    ######## Representacion Matricial de los T ####################
    ###############################################################
    
    a = []
    
    for u in range(0,n*n-1):
        
        a.append(tM(n,Lamda[u],Sim[u],k))
    
    return Lamda,a

In [32]:
def bra(A,B):
    
    return np.matmul(A,B)-np.matmul(B,A)

### Anticomutador


def Abra(A,B):
    
    return np.matmul(A,B)+np.matmul(B,A)

def f32(LamdL,n):
       
    u = range(0,n*n-1)
    
   # values = [list(v) for v in itertools.product(u, repeat=3)]
    di = {}
    
    for i,j,k in itertools.product(u, repeat=3):
        
        m = -(1j)*sp.Rational(1,4)*np.trace(np.matmul(LamdL[i],bra(LamdL[j],LamdL[k])))
        di[(i,j,k)] = m
        
    return di 

## Generadores $T_I$ 

En esta parte se calculan todos los $T_I$ en su representacion matricial. Por simplicidad aqui calculare todos los generadores para $n=3$,$n=4$,$n=5$. Para todos los valores de $2 \leq k < n$.

### n=3

In [33]:
# k=2

Lamda_3, T32 = tMf(3,2)

In [34]:
F_3 = f32(Lamda_3,3)

### n=4

In [35]:
# k = 2

Lamda_4, T42 = tMf(4,2) 

# k =3
T43 = tMf(4,3)[1]

In [36]:
F_4 = f32(Lamda_4,4)

### n = 5

In [37]:
# k = 2

Lamda_5, T52 = tMf(5,2)

# k = 3

T53 = tMf(5,3)[1]

# k = 4

T54 = tMf(5,4)[1]

In [38]:
F_5 = f32(Lamda_5,5)

### Pruebas de traza y algebra

### Algebra

In [39]:
def rhs3(I,J,Mat,estc,st):
    
    a = np.zeros((len(Mat[0]),len(Mat[0])))
    
    for i in range(0,len(Mat)):
        if st == 'l':
            p = 2j
        elif st == 't':
            p = 1j
        elif st == 'T':
            p = 1
        u = estc[(I,J,i)]*p*Mat[i]
      #  u[u!=0] = np.array([sp.nsimplify(i) for i in u[u!=0]],dtype=object)        
        a = a + u
        
    return a
    

In [40]:
def alg3(I,J,Mat,estc,st):
    
    a = rhs3(I,J,Mat,estc,st)
        
    b = bra(Mat[I],Mat[J])
    
    ## Verifiquemos que todo sea del mismo tipo
    
  #  a[a!=0] = np.array([sp.nsimplify(i) for i in a[a!=0]],dtype=object)
  #  b[b!=0] = np.array([sp.nsimplify(i) for i in b[b!=0]],dtype=object)
    
    if (np.absolute(b-a) < 1.0E-10).all():
        return True
    else:
        return False     

### Vamos ahora con las pruebas para cada caso

In [41]:
def Algtest3(Mat,estc,st):
    
    a = []
    
    for i,j in combinations(len(Mat),2):
        a.append(alg3(i,j,Mat,estc,st))
        
        # Para saber en que indices no se cumple
        
        #if alg2(n,i,j,Mat,estc,st) == False:
            
        #    print(i,j)
        
    if all(a) == True:
        return 'Todas las relaciones se cumplen'
    else:
        return 'Algo va mal'    

### Verificando las relaciones de conmutacion de los $\lambda_I$

In [42]:
Algtest3(Lamda_3,F_3,'l')

'Todas las relaciones se cumplen'

In [43]:
Algtest3(Lamda_4,F_4,'l')

'Todas las relaciones se cumplen'

In [44]:
Algtest3(Lamda_5,F_5,'l')

'Todas las relaciones se cumplen'

### Verificando las relaciones de commutacion de los $t_I$

In [45]:
Algtest3(T32,F_3,'t')

'Todas las relaciones se cumplen'

In [46]:
Algtest3(T42,F_4,'t')

'Todas las relaciones se cumplen'

In [47]:
Algtest3(T43,F_4,'t')

'Todas las relaciones se cumplen'

In [48]:
Algtest3(T52,F_5,'t')

'Todas las relaciones se cumplen'

In [49]:
Algtest3(T53,F_5,'t')

'Todas las relaciones se cumplen'

In [50]:
Algtest3(T54,F_5,'t')

'Todas las relaciones se cumplen'

### Verificando la traza

In [51]:
def checkTTmat(tmat,n,k):
    
    s = []
    
    rs = sp.Rational(np.math.factorial(n-2),2*np.math.factorial(k-1)*np.math.factorial(n-k-1))
    
    for i,j in itertools.product(range(0,len(tmat)), repeat=2):
        
        if abs(Traza(tmat[i],tmat[j]) - rs*Delta([i],[j])) < 1.0E-10:
            s.append(True)
        else:
            print(i,j)
            s.append(False)
    
    
    if all(s) == True:
        return 'Todas las relaciones se cumplen'
    else:
        return 'Algo va mal'

In [52]:
checkTTmat(T32,3,2)

'Todas las relaciones se cumplen'

In [53]:
checkTTmat(T42,4,2)

'Todas las relaciones se cumplen'

In [54]:
checkTTmat(T43,4,3)

'Todas las relaciones se cumplen'

In [55]:
checkTTmat(T52,5,2)

'Todas las relaciones se cumplen'

In [56]:
checkTTmat(T53,5,3)

'Todas las relaciones se cumplen'

In [57]:
checkTTmat(T54,5,4)

'Todas las relaciones se cumplen'

### Vamos ahora a calcular los $T_I$

En base a los cambios anteriores yo colocaria 

$(T_I)^{A}_{B} = i(t^t_I)^{A}_{B}$

In [58]:
def Ts(tm):
    
    s = tm[0].shape
    a = []
    
    for k in tm:
        
        u = np.array(np.zeros((s[0]+4,s[0]+4),dtype=object))
        u[4:,4:] = (1j)*np.transpose(k)
        a.append(u)
        
    return a   

In [59]:
TF32 = Ts(T32)
TF42 = Ts(T42)
TF43 = Ts(T43)
TF52 = Ts(T52)
TF53 = Ts(T53)
TF54 = Ts(T54)

In [60]:
Algtest3(TF32,F_3,'T')

'Todas las relaciones se cumplen'

In [61]:
Algtest3(TF42,F_4,'T')

'Todas las relaciones se cumplen'

In [62]:
Algtest3(TF43,F_4,'T')

'Todas las relaciones se cumplen'

In [63]:
Algtest3(TF52,F_5,'T')

'Todas las relaciones se cumplen'

In [64]:
Algtest3(TF53,F_5,'T')

'Todas las relaciones se cumplen'

In [65]:
Algtest3(TF54,F_5,'T')

'Todas las relaciones se cumplen'

## Verifiquemos la propiedad $T_I^\dagger = - T_I$

In [66]:
def anticheck(Matrix):
    
    a = []
    
    for i in Matrix:
        
        u = np.conjugate(np.transpose(i))
        
        if (u == -i).all():
            a.append(True)
        else:
            a.append(False)
            
    
    if all(a) == True:
        return 'Todas las relaciones se cumplen'
    else:
        return 'Algo va mal'            

In [67]:
anticheck(TF32)

'Todas las relaciones se cumplen'

In [68]:
anticheck(TF42)

'Todas las relaciones se cumplen'

In [69]:
anticheck(TF43)

'Todas las relaciones se cumplen'

In [70]:
anticheck(TF52)

'Todas las relaciones se cumplen'

In [71]:
anticheck(TF53)

'Todas las relaciones se cumplen'

In [72]:
anticheck(TF54)

'Todas las relaciones se cumplen'

In [73]:
def Gamma(n,k):
    
    d = TraDelta(n,k)
    
    u = np.array(np.zeros((d+4,d+4),dtype=object))

    for i in range(0,4): u[i][i] = 1
    
    for i in range(4,d+4): u[i][i] = -1#sp.Rational(d,np.math.factorial(k))
    
    return u
    

In [74]:
Gamma(4,2)

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, -1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, -1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, -1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, -1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, -1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, -1]], dtype=object)

In [75]:
def superTr(n,k,mat):
    
    s = Gamma(n,k)
    
    l = []
    
    rs = sp.Rational(np.math.factorial(n-2),2*np.math.factorial(k-1)*np.math.factorial(n-k-1))
    
    for i,j in itertools.product(range(0,n*n-1), repeat=2):
        
        t = np.matmul(s,np.matmul(mat[i],mat[j],dtype=object),dtype=object)
        
     #   t[t!=0] = np.array([sp.nsimplify(o) for o in t[t!=0]],dtype=object)
        
        u = np.trace(t)
        
        if abs((u - rs*Delta([i],[j]))) < 1.0E-10 :
            l.append(True)
        else:
            l.append(False)
            print(u,sp.nsimplify(rs*Delta([i],[j])))
    
    
    if all(l) == True:
        return 'Todas las relaciones se cumplen'
    else:
        return 'Algo va mal'
       
    return l        

In [76]:
superTr(3,2,TF32)

'Todas las relaciones se cumplen'

In [77]:
superTr(4,2,TF42)

'Todas las relaciones se cumplen'

In [78]:
superTr(4,3,TF43)

'Todas las relaciones se cumplen'

In [79]:
superTr(5,2,TF52)

'Todas las relaciones se cumplen'

In [80]:
superTr(5,3,TF53)

'Todas las relaciones se cumplen'

In [81]:
superTr(5,4,TF54)

'Todas las relaciones se cumplen'

### Las Cargas susy:

$(Q_{i'}^\alpha)^A_B = k! \Delta^A_{i'+4} \delta^\alpha_{B} \quad (\bar{Q}^{i'}_\alpha)^A_B = k!  \delta_\alpha^{A}\Delta_B^{i'+4}$

In [82]:
def char(d,k,a,i):
    
   # d = TraDelta(n,k)
    
    u = np.zeros((d+4,d+4),dtype=object)
    
    u[i+4][a] = np.math.factorial(k)*1
        
    return u
    
def charB(d,k,a,i):
    
   # d = TraDelta(n,k)
    
    u = np.zeros((d+4,d+4),dtype=object)
    
    u[a][i+4] = np.math.factorial(k)*1
        
    return u    

### Vamos a generarlas todas ahora

In [83]:
def Charges(n,k):
    
    d = TraDelta(n,k)
    p = {}
    
    for a in range(0,4):
        for i in range(0,d):
            
            p[(a,i)] = char(d,k,a,i)
            
    return p 

def ChargesB(n,k):
    
    d = TraDelta(n,k)
    p = {}
    
    for a in range(0,4):
        for i in range(0,d):
            
            p[(i,a)] = charB(d,k,a,i)
            
    return p    

In [84]:
Q32 = Charges(3,2)
QB32 = ChargesB(3,2)

In [85]:
Q32

{(0,
  0): array([[0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [2, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0]], dtype=object),
 (0,
  1): array([[0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [2, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0]], dtype=object),
 (0,
  2): array([[0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [2, 0, 0, 0, 0, 0, 0]], dtype=object),
 (1,
  0): array([[0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 2, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0]], dtype=object),
 (1,
  1): array([[0, 0, 0, 

In [86]:
Q42 = Charges(4,2)
QB42 = ChargesB(4,2)

Q43 = Charges(4,3)
QB43 = ChargesB(4,3)

In [87]:
Q52 = Charges(5,2)
QB52 = ChargesB(5,2)

Q53 = Charges(5,3)
QB53 = ChargesB(5,3)

Q54 = Charges(5,4)
QB54 = ChargesB(5,4)

### Los conmutadores 

$[T_I,Q^\alpha_{i'}] = i(t_I)_{i'}^{\ \ j'}Q^\alpha_{j'}$

$[T_I,\bar{Q}_\alpha^{i'}] = -i\bar{Q}_\alpha^{j'}(t_I)^{\ \ i'}_{j'}$

In [88]:
def rhsQ(n,k,Mat,tmat,I,i,a,st):
    
    d = TraDelta(n,k)
    
    p = np.zeros((d+4,d+4),dtype=object)
    
    for s in range(0,d):
        
        if st == 'Q':
            u = (tmat[I])[i][s]*Mat[(a,s)]
           # u[u!=0] = np.array([sp.nsimplify(i) for i in u[u!=0]],dtype=object)
        
            p = p + u
            
        elif st == 'QB':
            
            u = (tmat[I])[s][i]*Mat[(s,a)]
            #u[u!=0] = np.array([sp.nsimplify(i) for i in u[u!=0]],dtype=object)
        
            p = p + u
            
    if st == 'Q':
        return (1j)*p
        
    elif st == 'QB':
        return -(1j)*p

def algQ(n,k,Tmat,tmat,Mat,I,i,al,st):
    
    a = rhsQ(n,k,Mat,tmat,I,i,al,st)
    
    if st == 'Q':
        
        b = bra(Tmat[I],Mat[(al,i)])
    
    elif st == 'QB':
        
        b = bra(Tmat[I],Mat[(i,al)])
        
    ## Verifiquemos que todo sea del mismo tipo
    
    #a[a!=0] = np.array([sp.nsimplify(s) for s in a[a!=0]],dtype=object)
    #b[b!=0] = np.array([sp.nsimplify(s) for s in b[b!=0]],dtype=object)
    
    if (np.absolute(b - a) < 1.0E-10).all():
        return True
    else:
        return False 
    
def AlgtestQ(n,k,Tmat,tmat,Mat,st):
    
    a = []
    
    d = TraDelta(n,k)
    
    for I in range(0,n*n-1):
        for al in range(0,4):
            for i in range(0,d):
                a.append(algQ(n,k,Tmat,tmat,Mat,I,i,al,st))
    
        # Para saber en que indices no se cumple
        
        #if alg2(n,i,j,Mat,estc,st) == False:
            
        #    print(i,j)
        
    if all(a) == True:
        return 'Todas las relaciones se cumplen'
    else:
        return 'Algo va mal'    

In [89]:
AlgtestQ(3,2,TF32,T32,Q32,'Q')

'Todas las relaciones se cumplen'

In [90]:
AlgtestQ(3,2,TF32,T32,QB32,'QB')

'Todas las relaciones se cumplen'

In [91]:
AlgtestQ(4,2,TF42,T42,Q42,'Q')

'Todas las relaciones se cumplen'

In [92]:
AlgtestQ(4,2,TF42,T42,QB42,'QB')

'Todas las relaciones se cumplen'

In [93]:
AlgtestQ(4,3,TF43,T43,Q43,'Q')

'Todas las relaciones se cumplen'

In [94]:
AlgtestQ(4,3,TF43,T43,QB43,'QB')

'Todas las relaciones se cumplen'

In [95]:
AlgtestQ(5,2,TF52,T52,Q52,'Q')

'Todas las relaciones se cumplen'

In [96]:
AlgtestQ(5,2,TF52,T52,QB52,'QB')

'Todas las relaciones se cumplen'

In [97]:
AlgtestQ(5,3,TF53,T53,Q53,'Q')

'Todas las relaciones se cumplen'

In [98]:
AlgtestQ(5,3,TF53,T53,QB53,'QB')

'Todas las relaciones se cumplen'

In [99]:
AlgtestQ(5,4,TF54,T54,Q54,'Q')

'Todas las relaciones se cumplen'

In [100]:
AlgtestQ(5,4,TF54,T54,QB54,'QB')

'Todas las relaciones se cumplen'

### Supertraza de las cargas

In [101]:
def superTrQ(n,k,mat1,mat2):
    
    d = TraDelta(n,k)
    
    s = Gamma(n,k)
    
    l = []
    
#    rs = sp.Rational(np.math.factorial(n-2),2*np.math.factorial(k-1)*np.math.factorial(n-k-1))
    
    for i,j in itertools.product(range(0,d), repeat=2):
        for al,bl in itertools.product(range(0,4), repeat=2):
        
            t = np.matmul(s,np.matmul(mat1[(al,i)],mat2[(j,bl)],dtype=object),dtype=object)
        
           # t[t!=0] = np.array([sp.nsimplify(o) for o in t[t!=0]],dtype=object)
        
            u = np.trace(t)
        
            if abs(u + np.math.factorial(k)**2*Delta([al],[bl])*Delta([i],[j]))<1.0E-10:
                l.append(True)
            else:
                l.append(False)
                print(u,sp.nsimplify(-Delta([al],[bl])*Delta([i],[j])))
    
    if all(l) == True:
        return 'Todas las relaciones se cumplen'
    else:
        return 'Algo va mal'
       
    return l    

In [102]:
superTrQ(3,2,Q32,QB32)

'Todas las relaciones se cumplen'

In [103]:
superTrQ(4,2,Q42,QB42)

'Todas las relaciones se cumplen'

In [104]:
superTrQ(4,3,Q43,QB43)

'Todas las relaciones se cumplen'

In [105]:
superTrQ(5,2,Q52,QB52)

'Todas las relaciones se cumplen'

In [106]:
superTrQ(5,3,Q53,QB53)

'Todas las relaciones se cumplen'

In [107]:
superTrQ(5,4,Q54,QB54)

'Todas las relaciones se cumplen'

### Carga Central

$Z^A_B = iz(\delta^A_\alpha\delta_B^\alpha + 4 \Delta^A_{i'+4}\Delta^{i'+4}_B/d_n)$

In [108]:
def Z(n,k,c):
    
    d = TraDelta(n,k)
    
    u = np.zeros((d+4,d+4),dtype=object)
    
    for i in range(0,4):
        
        u[i,i] = (1j)*c
    
    for j in range(4,d+4):
        
        u[j,j] = (1j)*sp.Rational(4,d)*c
        
    return u     

### Relaciones de conmutacion

$[Z,Q^\alpha_{i'}] = iz(4/d_{n}-1)Q^\alpha_{i'}$

$[Z,\bar{Q}_\alpha^{i'}] = -iz(4/d_{n}-1)\bar{Q}_\alpha^{i'}$

In [109]:
def algQZ(n,k,Mat,i,al,st):
    
    d = TraDelta(n,k)
    
    if st == 'Q':
        
        a = (1j)*(4/d-1)*Mat[(al,i)]
        
        b = bra(Z(n,k,1),Mat[(al,i)])
    
    elif st == 'QB':
        
        a = -(1j)*(sp.Rational(4,d)-1)*Mat[(i,al)]
        
        b = bra(Z(n,k,1),Mat[(i,al)])   
        
    ## Verifiquemos que todo sea del mismo tipo
    
  #  a[a!=0] = np.array([sp.nsimplify(s) for s in a[a!=0]],dtype=object)
  #  b[b!=0] = np.array([sp.nsimplify(s) for s in b[b!=0]],dtype=object)
    
    if (np.absolute(b - a)<1.0E-10).all():
        return True
    else:
        print(np.absolute(b - a))
        return False 
    
def AlgtestQZ(n,k,Mat,st):
    
    a = []
    
    d = TraDelta(n,k)
    
    for al in range(0,4):
        for i in range(0,d):
            a.append(algQZ(n,k,Mat,i,al,st))
    
         #Para saber en que indices no se cumple
        
        if algQZ(n,k,Mat,i,al,st) == False:
            
            print(al,i)
        
    if all(a) == True:
        return 'Todas las relaciones se cumplen'
    else:
        return 'Algo va mal'    

In [110]:
AlgtestQZ(3,2,Q32,'Q')

'Todas las relaciones se cumplen'

In [111]:
AlgtestQZ(3,2,QB32,'QB')

'Todas las relaciones se cumplen'

In [112]:
AlgtestQZ(4,2,Q42,'Q')

'Todas las relaciones se cumplen'

In [113]:
AlgtestQZ(4,2,QB42,'QB')

'Todas las relaciones se cumplen'

In [114]:
AlgtestQZ(4,3,Q43,'Q')

'Todas las relaciones se cumplen'

In [115]:
AlgtestQZ(4,3,QB43,'QB')

'Todas las relaciones se cumplen'

In [116]:
AlgtestQZ(5,2,Q52,'Q')

'Todas las relaciones se cumplen'

In [117]:
AlgtestQZ(5,2,QB52,'QB')

'Todas las relaciones se cumplen'

In [118]:
AlgtestQZ(5,3,Q53,'Q')

'Todas las relaciones se cumplen'

In [119]:
AlgtestQZ(5,3,QB53,'QB')

'Todas las relaciones se cumplen'

In [120]:
AlgtestQZ(5,4,Q54,'Q')

'Todas las relaciones se cumplen'

In [121]:
AlgtestQZ(5,4,QB54,'QB')

'Todas las relaciones se cumplen'

### Generadores de $SU(d_n)$

1) Con los generadores no diagonales en los T_I generamos otros no diagonales como se muestra en el paper
2) Llenamos los lugares vacios en los T_I,
3) Buscamos los V_x que no estan generados por todos los anteriores y agregamos los ultimos diagonales

In [122]:
### Esta funcion devuelve los elementos de la representacion fundamental para los cuales Tr(VxTI) = 0 para todo I 

def NonV(n,k,ts):
    
    d = TraDelta(n,k)
    
    Fr, sm = GenLamda3(d)
    
    #######################################################################################################
    ##################### Calculo de las constantes y de que elementos de V_x no estan en los T_I #########
    #######################################################################################################
    
    num = 2*np.math.factorial(k-1)*np.math.factorial(n-k-1)
    dem = np.math.factorial(n-2)
    
    V_out = []
    V_dia = []
    
    for x in range(0,d*d-1):
        
        a = []
        
        for I in range(0,len(ts)):
            
            c = sp.Rational(num,dem)*np.trace(np.matmul(Fr[x],ts[I],dtype=object))
            
            a.append(c)
        
        if all([True if abs(k)<1.0E-10 else False for k in a]) == True:
            
            V_out.append(Fr[x])
            
        if (x>n*n-1) and (sm[x] == 'd'):
            V_dia.append(Fr[x])
                   
    return V_out + V_dia

############################################################################################################################
########## Ahora una funcion para trabajar con los elementos no diagonales #################################################

def nonT(n,k,ts):
    
    d = TraDelta(n,k)
    
    sm = GenLamda3(n)[1]
    
    nonT = []
    
    for i in range(0,len(ts)):
        
        if (sm[i] == 's' or sm[i] == 'as'):
            
            p = np.where(ts[i] != 0)
        
            p1 = p[0]
            p2 = p[1]
            
            u = [(s,o) for s,o in zip(p1,p2) if s<o]
        
            for l,m in u[:len(u)-1]:
            
                Ts = ts[i].copy()
                Ts[l][m] = -Ts[l][m]
                Ts[m][l] = -Ts[m][l]
            
                nonT.append(Ts)
            
    return nonT    


##########################################################################################################################
######################## LLenar Espacios Sim y AnSim #####################################################################

def nonTSim(n,k,ts):
    
    d = TraDelta(n,k)
    
    sm = GenLamda3(n)[1]
    
    u,v = np.zeros((d,d),dtype=object),np.zeros((d,d),dtype=object)
    
    for i in range(len(ts)):
                
        if sm[i] == 's':
            
            u = u + ts[i]
            
        elif sm[i] == 'as':
            
            v = v + ts[i]
    
    p1 = np.where(u==0) 
    p2 = np.where(v==0)
    
    NewS, NewAS = [],[]
    
    for l,m in zip(p1[0],p1[1]):
        
        if (l == m) or (l == d-1-m) or (l > m):
            continue
        else:
            M = np.zeros((d,d),dtype=object)
            M[l][m] = 1
            M[m][l] = M[l][m]
        
        NewS.append(M)
    
    for l,m in zip(p2[0],p2[1]):
        
        if (l == m) or (l == d-1-m) or (l > m):
            continue
        else:
            N = np.zeros((d,d),dtype=object)
            N[l][m] = 1j
            N[m][l] = -N[l][m]
        
        NewAS.append(N)
    
    return NewS + NewAS

In [123]:
##################################################################################################################
######################### Gram–Schmidt process ##################################################################

def Proj(A,B):
    
    u = np.trace(np.matmul(A,B,dtype=object))
    
    v = np.trace(np.matmul(B,B,dtype=object))
    
    return (u/v)*B

def newG(n,k,ts):
    
    u1 = nonT(n,k,ts)
    v1 = nonTSim(n,k,ts)
    z1 = NonV(n,k,ts + u1 + v1)
    
    tot = ts + u1 + v1 + z1
    
    return tot

def diagonelements(lista):
    
    dia = []
    
    for i in range(0,len(lista)):
        if (lista[i].diagonal() == 0).all():
            continue
        else:    
            dia.append(lista[i])
            
    return dia        

    
def GS(n,k,ts,st):
    
    se = newG(n,k,ts)
    
    VN = [se[0]]
    
    for i in range(1,len(se)):
        
        u = se[i]
        
        for j in range(0,i):
            
            u = u - Proj(se[i],VN[j])
        
        VN.append(u)
    
    # Normalizacion 
    
    if st == 'n':
        
    
        final = []
    
        dem = 2*np.math.factorial(k-1)*np.math.factorial(n-k-1)
        num = np.math.factorial(n-2)
    
        for i in range(len(VN)):
        
            s = np.trace(np.matmul(VN[i],VN[i],dtype=object))
        
            p = (num/dem)/s
        
            final.append((p**(0.5))*VN[i])    
        
        return final
    else: return VN

In [124]:
np.absolute(10j)

10.0

In [125]:
Proj(T42[0],T42[1])

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]], dtype=object)

### Otro enfoque

In [126]:
def GS2(n,k,ts,st):
    
    d = TraDelta(n,k)
    
    Fr = GenLamda3(d)[0]
    
    se = ts + Fr
    
    VN = [se[0]]
    
    for i in range(1,len(se)):
        
        u = se[i]
        
        for j in range(0,len(VN)):
            
            u = u - Proj(se[i],VN[j])
        
        if (np.absolute(u) < 1.0E-10).all():
            
            continue
            
            
        else: 
            
            VN.append(np.where(np.absolute(u)>1.0E-10,u,0*u))
    # Normalizacion 
    
    if st == 'n':
        
    
        final = []
    
        dem = 2*np.math.factorial(k-1)*np.math.factorial(n-k-1)
        num = np.math.factorial(n-2)
    
        for i in range(len(VN)):
        
            s = np.trace(np.matmul(VN[i],VN[i],dtype=object))
        
            p = sp.Rational(sp.Rational(num,dem),s)
        
            final.append((sp.sqrt(p))*VN[i])    
        
        return final
    else: return VN

In [127]:
def tracecc(lis):
    
    #final = newG(n,k,ts)
    
    dic = {}
    
    for i,j in itertools.product(range(0,len(lis)), repeat=2):
        
        if Traza(lis[i],lis[j]) != 0 and i != j:
            dic[(i,j)] = 1
            
    return dic

In [128]:
Ts32 = GS2(3,2,T32,'n')

In [129]:
Ts42 = GS2(4,2,T42,'n')

In [130]:
Ts43 = GS2(4,3,T43,'n')

In [131]:
Ts52 = GS2(5,2,T52,'n')

In [132]:
Ts53 = GS2(5,3,T53,'n')

In [133]:
Ts54 = GS2(5,4,T54,'n')

## Las constantes entre la representacion fundamental y la de los $T_I$

## La notacion es {[$V_y$],([$T_x$],[$C_{yx}$])}

In [134]:
def consta(n,k,ts):
    
    d = TraDelta(n,k)
    
    Fr = GenLamda3(d)[0]
    
    #######################################################################################################
    ##################### Calculo de las constantes y de que elementos de V_x no estan en los T_I #########
    #######################################################################################################
    
    dic_const = {}
    
    num = 2*np.math.factorial(k-1)*np.math.factorial(n-k-1)
    dem = np.math.factorial(n-2)
    
    for x in range(0,len(Fr)):
        
        a = []
        b = []
        
        for I in range(0,len(ts)):
            
            c = sp.Rational(num,dem)*np.trace(np.matmul(Fr[x],ts[I],dtype=object))
            
            if abs(c)>1.0E-10:
                
                a.append(c)
                b.append(I)
            
        dic_const[x] = (b,a)
            
    return dic_const

def verchech(lis1,lis2,ts,d):
    
    u = np.zeros((d,d),dtype=object)
    
    for i,j in zip(lis1,lis2):
        
        u = u + j*ts[i]
        
    return u     
        
def check(n,k,dic,ts):

    d = TraDelta(n,k)
    
    Fr = GenLamda3(d)[0] 
    
    a = []
    
    for i in dic:
        
        s = verchech((dic[i])[0],(dic[i])[1],ts,d)
        
        if (np.absolute(Fr[i] - s)<1.0E-10).all():
            
            a.append(True)
     
    if all(a):
        
        return 'Todo esta bien'
    else:
        return 'Algo esta mal'
        
    
    

In [135]:
d32 = consta(3,2,Ts32)

In [136]:
d32

{0: ([5], [2]),
 1: ([6], [2.00000000000000]),
 2: ([2, 7], [-1, sqrt(3)]),
 3: ([3], [-2]),
 4: ([4], [-2.00000000000000]),
 5: ([0], [2]),
 6: ([1], [2.00000000000000]),
 7: ([2, 7], [sqrt(3), 1])}

In [137]:
check(3,2,d32,Ts32)

'Todo esta bien'

In [138]:
d42 = consta(4,2,Ts42)

In [139]:
d42

{0: ([5, 15], [1, 1]),
 1: ([6, 16], [1.00000000000000, 1.00000000000000]),
 2: ([2, 7, 17], [-1/2, sqrt(3)/2, 1]),
 3: ([10, 18], [1, 1]),
 4: ([11, 19], [1.00000000000000, 1.00000000000000]),
 5: ([12, 20], [1, 1]),
 6: ([13, 21], [1.00000000000000, 1.00000000000000]),
 7: ([2, 7, 14, 22], [-sqrt(3)/6, -1/6, 2*sqrt(2)/3, 1]),
 8: ([3, 23], [-1, 1]),
 9: ([4, 24], [-1.00000000000000, 1.00000000000000]),
 10: ([0, 25], [1, 1]),
 11: ([1, 26], [1.00000000000000, 1.00000000000000]),
 12: ([27], [sqrt(2)]),
 13: ([28], [1.0*sqrt(2)]),
 14: ([2, 7, 14, 22], [5*sqrt(6)/12, 5*sqrt(2)/12, -1/3, sqrt(2)/2]),
 15: ([8, 29], [-1, 1]),
 16: ([9, 30], [-1.00000000000000, 1.00000000000000]),
 17: ([31], [sqrt(2)]),
 18: ([32], [1.0*sqrt(2)]),
 19: ([0, 25], [1, -1]),
 20: ([1, 26], [1.00000000000000, -1.00000000000000]),
 21: ([12, 20], [1, -1]),
 22: ([13, 21], [1.00000000000000, -1.00000000000000]),
 23: ([2, 7, 14, 17, 22],
  [sqrt(10)/4, -sqrt(30)/20, sqrt(15)/5, sqrt(10)/5, -sqrt(30)/10]),
 24

In [140]:
check(4,2,d42,Ts42)

'Todo esta bien'

In [141]:
d43 = consta(4,3,Ts43)

In [142]:
d43

{0: ([12], [2]),
 1: ([13], [2.00000000000000]),
 2: ([7, 14], [-2*sqrt(3)/3, 2*sqrt(6)/3]),
 3: ([10], [-2]),
 4: ([11], [-2.00000000000000]),
 5: ([5], [2]),
 6: ([6], [2.00000000000000]),
 7: ([2, 7, 14], [-2*sqrt(3)/3, 4/3, 2*sqrt(2)/3]),
 8: ([8], [2]),
 9: ([9], [2.00000000000000]),
 10: ([3], [-2]),
 11: ([4], [-2.00000000000000]),
 12: ([0], [2]),
 13: ([1], [2.00000000000000]),
 14: ([2, 7, 14], [2*sqrt(6)/3, 2*sqrt(2)/3, 2/3])}

In [143]:
check(4,3,d43,Ts43)

'Todo esta bien'

In [144]:
d52 = consta(5,2,Ts52)

In [145]:
d52

{0: ([5, 24], [2/3, 2*sqrt(2)/3]),
 1: ([6, 25],
  [0.666666666666667, 4.96705373128255e-9*sqrt(36028797018963978)]),
 2: ([2, 7, 26], [-1/3, sqrt(3)/3, 2*sqrt(2)/3]),
 3: ([10, 27], [2/3, 2*sqrt(2)/3]),
 4: ([11, 28],
  [0.666666666666667, 4.96705373128255e-9*sqrt(36028797018963978)]),
 5: ([12, 29], [2/3, 2*sqrt(2)/3]),
 6: ([13, 30],
  [0.666666666666667, 4.96705373128255e-9*sqrt(36028797018963978)]),
 7: ([2, 7, 14, 31], [-sqrt(3)/9, -1/9, 4*sqrt(2)/9, 2*sqrt(2)/3]),
 8: ([17, 32], [2/3, 2*sqrt(2)/3]),
 9: ([18, 33],
  [0.666666666666667, 4.96705373128255e-9*sqrt(36028797018963978)]),
 10: ([19, 34], [2/3, 2*sqrt(2)/3]),
 11: ([20, 35],
  [0.666666666666667, 4.96705373128255e-9*sqrt(36028797018963978)]),
 12: ([21, 36], [2/3, 2*sqrt(2)/3]),
 13: ([22, 37],
  [0.666666666666667, 4.96705373128255e-9*sqrt(36028797018963978)]),
 14: ([2, 7, 14, 23, 38],
  [-sqrt(6)/18, -sqrt(2)/18, -1/18, sqrt(15)/6, 2*sqrt(2)/3]),
 15: ([3, 39], [-2/3, 2*sqrt(2)/3]),
 16: ([4, 40],
  [-0.6666666666666

In [146]:
check(5,2,d52,Ts52)

'Todo esta bien'

In [147]:
d53 = consta(5,3,Ts53)

In [148]:
d53

{0: ([12, 24], [2/3, 2*sqrt(2)/3]),
 1: ([13, 25],
  [0.666666666666667, 4.96705373128255e-9*sqrt(36028797018963978)]),
 2: ([7, 14, 26], [-2*sqrt(3)/9, 2*sqrt(6)/9, 2*sqrt(2)/3]),
 3: ([19, 27], [2/3, 2*sqrt(2)/3]),
 4: ([20, 28],
  [0.666666666666667, 4.96705373128255e-9*sqrt(36028797018963978)]),
 5: ([21, 29], [2/3, 2*sqrt(2)/3]),
 6: ([22, 30],
  [0.666666666666667, 4.96705373128255e-9*sqrt(36028797018963978)]),
 7: ([7, 14, 23, 31], [-2/9, -sqrt(2)/9, sqrt(30)/9, 2*sqrt(2)/3]),
 8: ([10, 32], [-2/3, 2*sqrt(2)/3]),
 9: ([11, 33],
  [-0.666666666666667, 9.9341074625651e-9*sqrt(9007199254740993)]),
 10: ([5, 34], [2/3, 2*sqrt(2)/3]),
 11: ([6, 35], [0.666666666666667, 9.9341074625651e-9*sqrt(9007199254740993)]),
 12: ([36], [2*sqrt(3)/3]),
 13: ([37], [0.666666666666667*sqrt(3)]),
 14: ([2, 7, 14, 23, 31, 38],
  [-sqrt(6)/6, 7*sqrt(2)/18, 7/18, -sqrt(15)/18, 1/3, sqrt(5)/3]),
 15: ([17, 39], [-2/3, 2*sqrt(2)/3]),
 16: ([18, 40],
  [-0.666666666666667, 9.9341074625651e-9*sqrt(9007199

In [149]:
check(5,3,d53,Ts53)

'Todo esta bien'

In [150]:
d54 = consta(5,4,Ts54)

In [151]:
d54

{0: ([21], [2]),
 1: ([22], [2.00000000000000]),
 2: ([14, 23], [-sqrt(6)/2, sqrt(10)/2]),
 3: ([19], [-2]),
 4: ([20], [-2.00000000000000]),
 5: ([12], [2]),
 6: ([13], [2.00000000000000]),
 7: ([7, 14, 23], [-4/3, 5*sqrt(2)/6, sqrt(30)/6]),
 8: ([17], [2]),
 9: ([18], [2.00000000000000]),
 10: ([10], [-2]),
 11: ([11], [-2.00000000000000]),
 12: ([5], [2]),
 13: ([6], [2.00000000000000]),
 14: ([2, 7, 14, 23], [-sqrt(6)/2, 5*sqrt(2)/6, 5/6, sqrt(15)/6]),
 15: ([15], [-2]),
 16: ([16], [-2.00000000000000]),
 17: ([8], [2]),
 18: ([9], [2.00000000000000]),
 19: ([3], [-2]),
 20: ([4], [-2.00000000000000]),
 21: ([0], [2]),
 22: ([1], [2.00000000000000]),
 23: ([2, 7, 14, 23], [sqrt(10)/2, sqrt(30)/6, sqrt(15)/6, 1/2])}

In [152]:
check(5,4,d54,Ts54)

'Todo esta bien'

### Las versiones $(4+d_n) \times (4 + d_n)$

In [153]:
TFs32 = Ts(Ts32)
TFs42 = Ts(Ts42)
TFs43 = Ts(Ts43)
TFs52 = Ts(Ts52)
Tfs53 = Ts(Ts53)
Tfs54 = Ts(Ts54)

### Ultimas pruebas

In [154]:
checkTTmat(Ts32,3,2)

'Todas las relaciones se cumplen'

In [155]:
checkTTmat(Ts42,4,2)

'Todas las relaciones se cumplen'

In [156]:
checkTTmat(Ts43,4,3)

'Todas las relaciones se cumplen'

In [157]:
checkTTmat(Ts52,5,2)

'Todas las relaciones se cumplen'

In [158]:
checkTTmat(Ts53,5,3)

'Todas las relaciones se cumplen'

In [1]:
checkTTmat(Ts54,5,4)

NameError: name 'checkTTmat' is not defined

### Checks with S

In [160]:
def S(i,j):
    
    u = np.zeros((d+4,d+4),dtype=object)    
   # d = TraDelta(n,k)
     
    u[i+4][a] = np.math.factorial(k)*1
        
    return u

IndentationError: expected an indented block after 'for' statement on line 5 (3262312561.py, line 8)